Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [4]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data/real_data_layers.pkl")

In [14]:
# df = pd.DataFrame()
lr = 1e-5
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    for temperature in [0.07]:
        for dropout in [0.9, 0.8, 0.2, 0.5 
                       ]:
            for nb_genes in [500
                      ]:
                for layers in [ 
                    [200, 60, 60]
                    [200, 100, 100], [200, 40, 100], [200, 40, 40], [200, 40, 60]
                    [200, 100, 100, 100],[200, 80, 100], [100, 100], [100],
                    [200, 100, 80, 80],[200, 80, 80], [100, 80], [80],
                    [200, 100, 60, 60],[200, 80, 60], [100, 60], [60],
                    [200, 60, 40, 40],[200, 60, 40], [60, 40], [40],
                    [200, 60, 40, 20],[200, 60, 20], [60, 20], [20],
                              ]:
                    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
                    X = np.array(data_mat['X'])
                    Y = np.array(data_mat['Y'])
                    cluster_number = np.unique(Y).shape[0]

                    X = train.preprocess(X, nb_genes=nb_genes)

                    for run in range(3):
                        torch.manual_seed(run)
                        torch.cuda.manual_seed_all(run)
                        np.random.seed(run)
                        random.seed(run)
                        torch.backends.cudnn.deterministic = True
                        torch.backends.cudnn.benchmark = False
                        dresults = train.run(X,
                                             cluster_number,
                                             dataset,
                                             Y=Y,
                                             nb_epochs=30,
                                             lr=lr,
                                             temperature=temperature,
                                             dropout=dropout,
                                             evaluate=False,
                                             n_ensemble=1,
                                             layers=layers,
                                             save_to=f"{path}output/real_data/{run}/",
                                             save_pred = False)
                        dresults["temperature"] = temperature
                        dresults["dropout"] = dropout
                        dresults["lr"] = lr
                        dresults["layers"] = str(layers)
                        dresults["run"] = run
                        print(f".", end = "")
#                         print(f"# {temperature}, {dropout}, {lr}, {layers}", 
#                               dresults.get('COMBINED_kmeans_ari', ""),
#                               dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
#                               dresults.get('leiden_ari_0', ""))
                        df = df.append(dresults, ignore_index=True)

                        df.to_pickle(f"{path}output/pickle_results/real_data/real_data_layers.pkl")

>>>>> Data Quake_Smart-seq2_Trachea
SCZI  0.8291128219663276
(1350, 23341) (1350, 23341) keeping 500 genes
...(1350, 23341) (1350, 23341) keeping 500 genes
...(1350, 23341) (1350, 23341) keeping 500 genes
...(1350, 23341) (1350, 23341) keeping 500 genes
...>>>>> Data Quake_Smart-seq2_Diaphragm
SCZI  0.9596701189611787
(870, 23341) (870, 23341) keeping 500 genes
...(870, 23341) (870, 23341) keeping 500 genes
...(870, 23341) (870, 23341) keeping 500 genes
...(870, 23341) (870, 23341) keeping 500 genes
...>>>>> Data Quake_10x_Spleen
SCZI  0.909534824400645
(9552, 23341) (9552, 23341) keeping 500 genes
...(9552, 23341) (9552, 23341) keeping 500 genes
...(9552, 23341) (9552, 23341) keeping 500 genes
...(9552, 23341) (9552, 23341) keeping 500 genes
...>>>>> Data Young
SCZI  0.6629811886776039
(5685, 33658) (5685, 33658) keeping 500 genes
...(5685, 33658) (5685, 33658) keeping 500 genes
...(5685, 33658) (5685, 33658) keeping 500 genes
...(5685, 33658) (5685, 33658) keeping 500 genes
...>>>>> 

In [15]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")
sczi = sczi[sczi["dataset"].isin(files)]
sczi.mean()

ARI       0.745684
NMI       0.777119
sil       0.070277
run       1.000000
time    195.854597
dtype: float64

In [16]:
# df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_layers.pkl")
df = df[df["dataset"].isin(files)]
r = df.groupby(["temperature", "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "lr"]).T
r

temperature                       0.07                              
dropout                            0.2       0.5       0.8       0.9
layers               lr                                             
[100, 100]           0.00001  0.347035  0.508908  0.691095  0.728694
[100, 60]            0.00001  0.351340  0.510541  0.695639  0.712584
[100, 80]            0.00001  0.363358  0.501141  0.674966  0.730938
[100]                0.00001  0.347455  0.550813  0.675218  0.714603
[200, 100, 100, 100] 0.00001  0.283721  0.490465  0.664494  0.728761
[200, 100, 100]      0.00001  0.308330  0.484588  0.698169  0.723520
[200, 100, 60, 60]   0.00001  0.273495  0.502064  0.688410  0.717062
[200, 100, 80, 80]   0.00001  0.280067  0.493029  0.679508  0.712243
[200, 40, 100]       0.00001  0.337072  0.503244  0.681031  0.725894
[200, 40, 40]        0.00001  0.321346  0.514351  0.690546  0.719508
[200, 40, 60]        0.00001  0.320159  0.515770  0.679362  0.736921
[200, 60, 20]        0.00001  0.326169  0.503053  0.691110  0.722476
[200, 60, 40, 20]    0.00001  0.315606  0.515720  0.683829  0.703756
[200, 60, 40, 40]    0.00001  0.278845  0.501156  0.670244  0.699728
[200, 60, 40]        0.00001  0.314157  0.505045  0.696995  0.720271
[200, 60, 60]        0.00001  0.315512  0.501306  0.681699  0.717035
[200, 80, 100]       0.00001  0.313856  0.497448  0.680231  0.727390
[200, 80, 60]        0.00001  0.316093  0.487865  0.683991  0.723043
[200, 80, 80]        0.00001  0.313540  0.497773  0.693930  0.725029
[20]                 0.00001  0.358403  0.561932  0.713330  0.720109
[40]                 0.00001  0.367622  0.551808  0.667143  0.729160
[60, 20]             0.00001  0.343458  0.518456  0.688198  0.705166
[60, 40]             0.00001  0.356521  0.510916  0.692391  0.723568
[60]                 0.00001  0.362298  0.540747  0.678830  0.719912
[80]                 0.00001  0.355006  0.550142  0.670740  0.719079

In [13]:
r.mean(axis = 1)

layers                lr     
[100, 100]            0.00001    0.568933
[100, 60]             0.00001    0.567526
[100, 80]             0.00001    0.567601
[100]                 0.00001    0.572022
[200, 100, 100, 100]  0.00001    0.541860
[200, 100, 100]       0.00001    0.553652
[200, 100, 60, 60]    0.00001    0.545258
[200, 100, 80, 80]    0.00001    0.541212
[200, 40, 100]        0.00001    0.561810
[200, 40, 40]         0.00001    0.561438
[200, 40, 60]         0.00001    0.563053
[200, 60, 20]         0.00001    0.560702
[200, 60, 40, 20]     0.00001    0.554728
[200, 60, 40, 40]     0.00001    0.537493
[200, 60, 40]         0.00001    0.559117
[200, 80, 100]        0.00001    0.554731
[200, 80, 60]         0.00001    0.552748
[200, 80, 80]         0.00001    0.557568
[20]                  0.00001    0.588443
[40]                  0.00001    0.578933
[60, 20]              0.00001    0.563820
[60, 40]              0.00001    0.570849
[60]                  0.00001    0.575447
[80]

In [ ]:
# df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "lr"]).max()

In [ ]:
# r = df[(df["layers"]=="[40]")
#        &(df["temperature"]==0.01)
#        &(df["lr"]==1e-5)
#        &(df["dropout"]==0.9)
#       ]
r = df[(df["layers"]=="[200, 100, 30, 30]")
       &(df["temperature"]==0.07)
       &(df["lr"]==1e-5)
       &(df["dropout"]==0.9)
      ]


r.mean()

In [ ]:
comb = pd.merge(r, sczi, on=["dataset", "run"])[[
    "dataset", "kmeans_ari_0", "ARI", "kmeans_nmi_0", "NMI"
]].rename(columns = {"kmeans_ari_0": "contrative-sc", "ARI": "sczi"})
comb

In [ ]:
comb =pd.melt(comb, id_vars=["dataset"], value_vars=["contrative-sc", "sczi"])

In [ ]:
comb.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot( data = comb, y="value", x = "variable")
plt.xticks(rotation = 90)

In [ ]:
sns.barplot(x = "dataset", data = comb, y="value", hue = "variable")
plt.xticks(rotation = 90)

In [ ]:
df.groupby("dataset").mean()

# Importance of input size

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_genes", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    for run in range(2):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        for nb_genes in [100, 200, 500, 1000, 1500, 2000, 5000, 8000]:

            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            cluster_number = np.unique(Y).shape[0]

            X = train.preprocess(X, nb_genes=nb_genes)
            nb_zeros = int(0.8 * nb_genes)
            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=100,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/inputs/{dataset}_{nb_genes}/")

            #         df.loc[df.shape[0]] = [
            #                 dataset, perc_0, nb_genes, 'kmeans_representation_0',dresults['kmeans_representation_0']
            #             ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, nb_genes, 'leiden_representation_0',
                dresults['leiden_representation_0'], run]

#             pxt = PCA(2).fit_transform(X)
#             dresults["original"] = utils.evaluate(X, Y, cluster_number)[1]
#             dresults["pca"] = utils.evaluate(pxt, Y, cluster_number)[1]
            print(dresults)
    df.to_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df.groupby(["nb_genes"]).mean()

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_genes": "Nb input genes"})

In [ ]:
df["dataset"].unique()


In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size.pdf", bbox_inches='tight')

In [ ]:
datasets = ['10X PBMC (2100)',
       'Mouse ES\nCell (2100)', 'Worm Neuron\nCell (2100)',
       'Mouse Bladder\n Cell (2100)']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size_2100.pdf", bbox_inches='tight')

# Importance of the number of epochs

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_epochs", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    nb_genes = 1500
    for epochs in [5, 50, 100, 300]:

        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
        print(f"Perc 0 {perc_0}")
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)
        nb_zeros = int(0.8 * nb_genes)
        for run in range(2):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=epochs,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/epochs/{dataset}_{epochs}/")

            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'kmeans_representation_0',
                dresults['kmeans_representation_0'], run
            ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'leiden_representation_0',
                dresults['leiden_representation_0'], run
            ]

            print(dresults)
            df.to_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_epochs": "Nb epochs"})

In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(7, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb epochs",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title ="Number of epochs",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_nb_epochs.pdf", bbox_inches='tight')